In [0]:
%pip install openpyxl
%pip install unidecode

In [0]:
import pandas as pd

excel_path = "test.xlsx"
df_pd = pd.read_excel(excel_path) 

df_pd_cleaned = df_pd.astype(str).fillna("") # convert all columns to string

df = spark.createDataFrame(df_pd_cleaned)
df.display()

In [0]:
# We need to have some way of handling column names. I prefer to do it automatically.
import re
from unidecode import unidecode

def camel_to_snake(name: str) -> str:
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def normalize_spark_columns_names(df):
    new_column_names = []

    for col_name in df.columns:
        # camelCase -> snake_case
        col_name = camel_to_snake(col_name)
        # Espaços para underline
        col_name = col_name.replace(' ', '_')
        # Somente letras/números/_ com regex
        col_name = re.sub(r'\W+', '_', col_name)
        # Remover acentuação
        col_name = unidecode(col_name)
        # Tudo minúsculo (redundante, mas seguro)
        col_name = col_name.lower()
        new_column_names.append(col_name)

    # Renomear colunas no DataFrame
    for old_name, new_name in zip(df.columns, new_column_names):
        df = df.withColumnRenamed(old_name, new_name)

    return df

In [0]:
df = normalize_spark_columns_names(df)
df.display()

In [0]:
df.columns

In [0]:
%sql
create schema staging

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("staging.test_customer_data")

In [0]:
%sql
select * from staging.test_customer_data